In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

In [ ]:
def find_delivery_time(list_of_info):
    time = 'Not Found'
    for info in list_of_info:
        if "Min" in info:
            time = info
    return time

def find_delivery_fee(list_of_info):
    fee = 'Not Found'
    for info in list_of_info:
        if "Delivery Fee" in info:
            fee = info
    return fee

def rewards(x):
    return True if x == 'Buy 1, Get 1 Free' else False

class RestaurantInfo:
    
    def __init__(self, info, link):
        self.deal = info[0]
        self.name = info[1]
        self.link = link
        self.fee = find_delivery_fee(info)
        self.time = find_delivery_time(info)
    
    @staticmethod
    def is_satisfied(info):
        return rewards(info[0])
    
    def print_info(self):
        print('The restaurant {0} has deal {1} with fee {2} and delivery time {3}'.format(self.name, self.deal, self.fee, self.time))

In [ ]:
def get_all_restaurants(driver, my_location, category = 'Deals'):
    '''
    None = get_to_page(driver, my_location, category = 'Deals')
    
    Perform a series of simulated clicks on website to get to desired Uber Eats page.
    
    @param driver: WebDriver
        WebDriver object from Selenium.
    @param my_location: str
        User location input on ubereats.com.
    @param category: str
        The category user want to check.
    '''
    # Wait 10 seconds maximum to locate the web elements.
    location_pos = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="location-typeahead-home-input"]'))
    )
    find_food_pos = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="main-content"]/div[1]/div[2]/div/div[1]/button'))
    )
    
    # Perform series of actions to type my location in the text box
    # and redirect to the new page.
    actions = ActionChains(driver)
    actions.send_keys_to_element(location_pos, my_location)
    actions.pause(1)
    actions.click(find_food_pos)
    actions.perform()
    
    # If the category can be found in the pre_defined dictionary,
    # use the value in the dictionary as xpath.
    if category == 'Deals':
        nav_pos = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, 'nav'))
        )
        deals_pos = nav_pos.find_element_by_tag_name("li")
        deals_pos.click()
    else:
        search_pos = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'search-suggestions-typeahead-input'))
        )
        actions = ActionChains(driver)
        actions.send_keys_to_element(search_pos, category)
        actions.key_down(Keys.ENTER).key_up(Keys.ENTER)
        actions.perform()
    
    # Wait until all elements are present in the list of restaurants.
    list_of_restaurants = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, '//*[@id="main-content"]/div/div/div[2]/div/div[2]'))
    )
    list_of_restaurants = list_of_restaurants[0]
    
    # Use the fact that all restaurants have only one figure to find number of restaurants in the page.
    figures = WebDriverWait(driver, 10).until(
        EC.visibility_of_all_elements_located((By.TAG_NAME, 'figure'))
    )
    
    num_of_restaurants = len(figures)
    restaurants = []
    
    for i in range(num_of_restaurants):
        WebDriverWait(driver, 10).until(
            EC.text_to_be_present_in_element((By.XPATH, '//*[@id="main-content"]/div/div/div/div/div[2]/div[{0}]'.format(i + 1)),
                                          ' ')
        )
        item = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="main-content"]/div/div/div/div/div[2]/div[{0}]'.format(i + 1)))
        )
        item_info = item.text.split('\n')
        link_info = item.find_element_by_tag_name('a')
        link = link_info.get_attribute('href')
        
        if RestaurantInfo.is_satisfied(item_info):
            restaurants.append(RestaurantInfo(item_info, link))
    return restaurants

In [ ]:
driver = webdriver.Chrome('C:\WebDriver\chromedriver.exe')
driver.maximize_window()
driver.get('https://www.ubereats.com')

get = get_all_restaurants(driver, '181 woodsworth road', 'Bubble Tea')
for res in get:
    res.print_info()

In [ ]:
for res in get:
    print(res.link)

In [ ]:
test = [1, 2, 3, 4, 5]

for i in range(10):
    print('i = {0}'.format(i))
    try:
        print(test[i])
    except:
        continue

In [ ]:
import gunicorn
gunicorn.__version__

In [6]:
import requests
from lxml import html

r = requests.get('https://www.ubereats.com/ca/toronto/food-delivery/shuyi-tealicious-%E4%B9%A6%E4%BA%A6%E7%83%A7%E4%BB%99%E8%8D%89-north-york/Ww473q4YTyGr4XJ1oCQTaA?pl=JTdCJTIyYWRkcmVzcyUyMiUzQSUyMjE4MSUyMFdvb2Rzd29ydGglMjBSZCUyMiUyQyUyMnJlZmVyZW5jZSUyMiUzQSUyMjI3NmY5NDIwLTFlYmMtNjAyOS1kNTdlLTc5MGRhNmU1MWRmZSUyMiUyQyUyMnJlZmVyZW5jZVR5cGUlMjIlM0ElMjJ1YmVyX3BsYWNlcyUyMiUyQyUyMmxhdGl0dWRlJTIyJTNBNDMuNzY0Njc0OSUyQyUyMmxvbmdpdHVkZSUyMiUzQS03OS4zNzIxMTA5JTdE')
tree = html.fromstring(r.content)
test = tree.xpath('//*[@id="main-content"]/div[3]/ul/li[1]/h2/span/text()')

In [27]:
for i in grid[0].getchildren():
    string = i.text_content()
    idx = string.find('$')
    print(string[:idx - 2])
    print(string)

Red Bull Energy Drink 250mL 4 Pack
Red Bull Energy Drink 250mL 4 PackCA$8.88
Jack Link's Original Beef Jerky 230g
Jack Link's Original Beef Jerky 230gCA$15.99
Ben and Jerry's Half Baked 500mL
Ben and Jerry's Half Baked 500mLCA$7.99


In [2]:
import requests
from lxml import html

current_url = 'https://www.ubereats.com/ca/toronto/food-delivery/shuyi-tealicious-%E4%B9%A6%E4%BA%A6%E7%83%A7%E4%BB%99%E8%8D%89-northtown-way/HgI2QkOiS5KOwH0aA4_PRg?pl=JTdCJTIyYWRkcmVzcyUyMiUzQSUyMjE4MSUyMFdvb2Rzd29ydGglMjBSZCUyMiUyQyUyMnJlZmVyZW5jZSUyMiUzQSUyMjI3NmY5NDIwLTFlYmMtNjAyOS1kNTdlLTc5MGRhNmU1MWRmZSUyMiUyQyUyMnJlZmVyZW5jZVR5cGUlMjIlM0ElMjJ1YmVyX3BsYWNlcyUyMiUyQyUyMmxhdGl0dWRlJTIyJTNBNDMuNzY0Njc0OSUyQyUyMmxvbmdpdHVkZSUyMiUzQS03OS4zNzIxMTA5JTdE'
r = requests.get(current_url)
tree = html.fromstring(r.content)
current_deal = tree.xpath('//*[@id="main-content"]/div[3]/ul/li[1]/h2/span/text()')
print(current_deal)

['Buy 1, Get 1 Free']


In [4]:

current_url = 'https://www.ubereats.com/ca/toronto/food-delivery/coco-fresh-tea-%26-juice-7398-yonge-st/4G9bwp9IQ46trzDKC9kcDA?pl=JTdCJTIyYWRkcmVzcyUyMiUzQSUyMjE4MSUyMFdvb2Rzd29ydGglMjBSZCUyMiUyQyUyMnJlZmVyZW5jZSUyMiUzQSUyMjI3NmY5NDIwLTFlYmMtNjAyOS1kNTdlLTc5MGRhNmU1MWRmZSUyMiUyQyUyMnJlZmVyZW5jZVR5cGUlMjIlM0ElMjJ1YmVyX3BsYWNlcyUyMiUyQyUyMmxhdGl0dWRlJTIyJTNBNDMuNzY0Njc0OSUyQyUyMmxvbmdpdHVkZSUyMiUzQS03OS4zNzIxMTA5JTdE'
r = requests.get(current_url)
tree = html.fromstring(r.content)
current_deal = tree.xpath('//*[@id="main-content"]/div[3]/ul/li[1]/h2/span/text()')    
print(current_deal)

['Buy 1, Get 1 Free']


In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

driver = webdriver.Chrome('C:\WebDriver\chromedriver.exe')
driver.maximize_window()
driver.get('https://www.ubereats.com/ca/toronto/food-delivery/coco-fresh-tea-%26-juice-7398-yonge-st/4G9bwp9IQ46trzDKC9kcDA?pl=JTdCJTIyYWRkcmVzcyUyMiUzQSUyMjE4MSUyMFdvb2Rzd29ydGglMjBSZCUyMiUyQyUyMnJlZmVyZW5jZSUyMiUzQSUyMjI3NmY5NDIwLTFlYmMtNjAyOS1kNTdlLTc5MGRhNmU1MWRmZSUyMiUyQyUyMnJlZmVyZW5jZVR5cGUlMjIlM0ElMjJ1YmVyX3BsYWNlcyUyMiUyQyUyMmxhdGl0dWRlJTIyJTNBNDMuNzY0Njc0OSUyQyUyMmxvbmdpdHVkZSUyMiUzQS03OS4zNzIxMTA5JTdE')

WebDriverWait(driver, 10).until(
                EC.text_to_be_present_in_element(
                    (By.XPATH, '//*[@id="main-content"]/div[3]/ul/li[1]/h2/span'
                ), ' ')
            )
current_deal = driver.find_element_by_xpath('//*[@id="main-content"]/div[3]/ul/li[1]/h2/span')
print(current_deal)

InvalidSelectorException: Message: invalid selector: The result of the xpath expression "//*[@id="main-content"]/div[3]/ul/li[1]/h2/span/text()" is: [object Text]. It should be an element.
  (Session info: chrome=89.0.4389.82)


In [8]:
current_deal = driver.find_element_by_xpath('//*[@id="main-content"]/div[3]/ul/li[1]/h2/span')
print(current_deal.text)

Buy 1, Get 1 Free


In [9]:
grid = driver.find_element_by_xpath('//*[@id="main-content"]/div[3]/ul/li[1]/ul')
print(grid)

<selenium.webdriver.remote.webelement.WebElement (session="71191402aa01da0085591bae1b661d38", element="128990dd-2850-4682-b860-818bb07906d2")>


In [13]:
item = grid.find_elements_by_tag_name('li')[0]

In [14]:
item.text

'Jasmine Milk Tea\n$5.15'